## 3.3 Compare defect environment in the MLFF and DFT relaxed structures

### 3.3.1 Compare the MLFF & DFT structures obtained with the same distortion

In [3]:
import pickle

# read the files
with open('../data/Test_data/result_structure.pkl','rb') as f:
    structures = pickle.load(f)
with open('../data/Test_data/result_energies.pkl','rb') as f:
    energies = pickle.load(f)

In [4]:
import numpy as np
from copy import copy
import warnings
from numpy.typing import ArrayLike, NDArray

from pymatgen.core.structure import Structure
from pymatgen.analysis.structure_matcher import StructureMatcher
from pymatgen.io.ase import AseAtomsAdaptor

from dscribe.descriptors import SOAP

from shakenbreak.input import identify_defect

def get_local_structs(
    struct_dft, struct_ff, coords_dft, coords_ff, r=3.5
) -> list:
    """Get atoms within a given radius (r, in A) around the site with fractional
    coordinates `coords`. Return a list of the "local" structures for the two
    input structures (struct_dft, struct_ff).
    """
    s_dft = struct_dft.copy()
    s_ff = struct_ff.copy()
    # Find defect site
    s_dft.append("V", coords_dft, coords_are_cartesian=False)
    s_ff.append("V", coords_ff, coords_are_cartesian=False)

    # Sites within radius
    cutoff = copy(r)
    cart_coords_defect_dft = np.array(coords_dft) @ np.array(s_dft.lattice.matrix)
    cart_coords_defect_ff = np.array(coords_ff) @ np.array(s_dft.lattice.matrix)
    sites_dft = s_dft.get_sites_in_sphere(cart_coords_defect_dft, cutoff)
    sites_ff = s_ff.get_sites_in_sphere(cart_coords_defect_ff, cutoff)
    # Make sure same number of sites in FF and DFT local structs
    while not len(sites_dft) == len(sites_ff) and cutoff < 7.0:
        # warnings.warn(
        #     f"Local structures with r {cutoff} A are not the same size! "
        #     + "Try increasing the radius. "
        #     + f"ff: {len(sites_dft)}, dft: {len(sites_ff)}"
        # )
        cutoff += 1.0
        sites_dft = s_dft.get_sites_in_sphere(cart_coords_defect_dft, cutoff)
        sites_ff = s_ff.get_sites_in_sphere(cart_coords_defect_ff, cutoff)

    if cutoff < 7.0:
        structs = []
        for s, sites in zip([s_dft, s_ff], [sites_dft, sites_ff]):
            structs.append(
                Structure(
                    lattice=s.lattice,
                    species=[i.species for i in sites],
                    coords=[i.coords for i in sites],
                    coords_are_cartesian=True,
                )
            )
    else:  # If cutoff is too large, return original structures
        structs = [s_dft, s_ff]

    return structs


def get_local_struct(
    struct, coords, r=3.5
) -> Structure:
    """Get atoms within a 3.5 A radius of a site with fractional
    coordinates `frac`.
    """
    s = struct.copy()
    # Find defect site
    s.append("V", coords, coords_are_cartesian=False)

    # Sites within radius
    cutoff = copy(r)
    cart_coords_defect = np.array(coords) @ np.array(s.lattice.matrix)
    sites = s.get_sites_in_sphere(cart_coords_defect, cutoff)

    return Structure(
        lattice=s.lattice,
        species=[i.species for i in sites],
        coords=[i.coords for i in sites],
        coords_are_cartesian=True,
    )


def _calculate_atomic_disp(
    struct1: Structure,
    struct2: Structure,
    stol: float = 0.5,
    ltol: float = 0.3,
    angle_tol: float = 5,
) -> tuple:
    """
    Calculate root mean square displacement and atomic displacements,
    normalized by the free length per atom ((Vol/Nsites)^(1/3)) between
    two structures.

    Args:
        struct1 (:obj:`Structure`):
            Structure to compare to struct2.
        struct2 (:obj:`Structure`):
            Structure to compare to struct1.
        stol (:obj:`float`):
            Site tolerance used for structural comparison (via
            `pymatgen`'s `StructureMatcher`), as a fraction of the
            average free length per atom := ( V / Nsites ) ** (1/3). If
            output contains too many 'NaN' values, this likely needs to
            be increased.
            (Default: 0.5)

    Returns:
        :obj:`tuple`:
            Tuple of normalized root mean squared displacements and
            normalized displacements between the two structures.
    """
    sm = StructureMatcher(
        ltol=ltol, stol=stol, angle_tol=angle_tol, primitive_cell=False, scale=True
    )
    struct1, struct2 = sm._process_species([struct1, struct2])
    struct1, struct2, fu, s1_supercell = sm._preprocess(struct1, struct2)
    match = sm._match(
        struct1, struct2, fu, s1_supercell, use_rms=True, break_on_match=False
    )

    if match is None:
        return None
    return match[0], match[1]


def calc_rms_1_struct(
    s1, s2, stol=0.6, ltol=0.5, angle_tol=6, min_dist=0.1, metric="disp"
):
    """Calculate structural difference between 2 structures."""
    ref_structure = s1
    normalization = (len(ref_structure) / ref_structure.volume) ** (1 / 3)
    try:
        _, norm_dist = _calculate_atomic_disp(
            struct1=s1,
            struct2=s2,
            stol=stol,
            ltol=ltol,
            angle_tol=angle_tol,
        )
        norm_dist = norm_dist / normalization
        if metric == "disp":
            # Remove normalization:
            disp = (
                np.sum(norm_dist[norm_dist > min_dist])
            )  # Only include displacements above min_dist threshold,
        elif metric == "max_dist":
            disp = (
                max(norm_dist)
            )
        else:
            raise ValueError(
                f"Invalid metric '{metric}'. Must be one of 'disp' or "
                f"'max_dist'."
            )
        return disp
    except TypeError:
        warnings.warn(
            f"pymatgen StructureMatcher could not match lattices for structures."
        )



def get_soap_vec(
    struct: Structure,
    r_cut=5, n_max=8, l_max=6, sigma=0.2,
    DUMMY_SPECIES = "Si"
) -> NDArray:
    """Get the SOAP vector for each site in the structure.

    Args:
        struct: Structure object to compute the SOAP vector for

    Returns:
        NDArray: SOAP vector for each site in the structure,
            shape (n_sites, n_soap_features)
    """
    adaptor = AseAtomsAdaptor()
    species_ = [str(el) for el in struct.composition.elements]
    dummy_structure = struct.copy()
    for el in species_:
        dummy_structure.replace_species({str(el): DUMMY_SPECIES})
    soap_desc = SOAP(species=[DUMMY_SPECIES], r_cut=r_cut, n_max=n_max, l_max=l_max, sigma=sigma, periodic=True)
    # average="inner"
    vecs = soap_desc.create(adaptor.get_atoms(dummy_structure))
    return vecs


def cosine_similarity(vec1, vec2) -> float:
    """Cosine similarity between two vectors."""
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


def cosine_distance(a, b):
    """Cosine distance of two vectors."""
    return 1 - cosine_similarity(a, b)


def indentify_def_cor(defect_structure, bulk_structure):
    defect_object = identify_defect(defect_structure, bulk_structure)
    defect_frac_coords = defect_object.site.frac_coords
    return(defect_frac_coords)

#### 3.3.2 Compare structures using the root mean squared displacements

A quick way to compare two structures is the root mean squared displacements (https://en.wikipedia.org/wiki/Root-mean-square_deviation_of_atomic_positions). Esentially, we map the two structures so that their sites are aligned, and then we calculate the distance between the aligned sites. The smaller the distance, the more similar the structures. The distances between the aligned sites are then averaged to get the root mean squared displacement (RMS).

The code below will compare the ground state (i.e. most favourable) structure obtained from the DFT relaxations (our ground truth) with the structure obtained for the same distortion with the MLFF relaxation.

In [5]:
from pprint import pprint

In [8]:
from pymatgen.io.vasp import Poscar

# Load the POSCAR file
poscar = Poscar.from_file("POSCAR")
# Access the structure from the POSCAR object
structure_perfect = poscar.structure

In [9]:
# Cutoff for local structures
# r_cutoff = 3.5
min_dist = 0.1

results = {}
for comp in energies:
    print("Composition", comp)
    for defect in energies[comp]:
        print(defect)
        # Get distortion with lowest energy
        d = energies[comp][defect]["DFT"]["distortions"]
        # print(energies[comp][defect]["DFT"]["D_Unperturbed_Defect"])
        d["Unperturbed"] = energies[comp][defect]["DFT"]["Unperturbed"]
        sorted_energies = dict(sorted(d.items(), key=lambda item: item[1]))
        gs_dist = list(sorted_energies.keys())[0]
        print("Ground state:", gs_dist, round(sorted_energies[gs_dist], 1))
        # Get lowest energy structure obtained with DFT
        gs_struct_dft = structures[comp][defect]["DFT"][gs_dist]
        # Get final structure obtained with FF for the same distortion as DFT
        gs_struct_ff = structures[comp][defect]["MLFF"][gs_dist]
        # Get defect coordinates
        coords_dft = indentify_def_cor(gs_struct_dft, structure_perfect)
        coords_mlff = indentify_def_cor(gs_struct_ff, structure_perfect)
        # Get local structs (not needed)
        # s1, s2 = get_local_structs(gs_struct_dft, gs_struct_ff, coords, r=r_cutoff)

        # Calculate root mean squared displacement
        rms = calc_rms_1_struct(
            gs_struct_dft, gs_struct_ff,
            metric="disp", # "disp" calculates the root mean squared displacement
            # Parameters for the structural comparison:
            stol=1.2, ltol=0.6, min_dist=min_dist
        )
        # And the maximum distance (in Angstroms) between the aligned sites of the 2 structures
        max_dist = calc_rms_1_struct(
            gs_struct_dft, gs_struct_ff,
            metric="max_dist", # "max_dist" calculates the maximum distance between the aligned sites
            # Parameters for the structural comparison:
            stol=1.2, ltol=0.6, min_dist=min_dist
        )

        results[defect] = {
            "Defect": defect,
            "Host": comp,
            "GS distortion": gs_dist,
            "\u0394 E (eV)": round(sorted_energies[gs_dist], 1),
            "Local RMS (A)": round(rms, 1) if rms is not None else None,
            "Max. dist. (A)": round(max_dist, 1) if max_dist is not None else None,
        }

Composition Sb2S3
Sb_S/Sb_S1
Ground state: D_-40.0%_BDM_Distortion -0.0
Sb_S/Sb_S3
Ground state: D_10.0%_BDM_Distortion -0.0
Sb_S/Sb_S2
Ground state: D_10.0%_BDM_Distortion -0.0
Int_Sb/Int_Sb3
Ground state: D_10.0%_BDM_Distortion -0.7
Int_Sb/Int_Sb4
Ground state: D_60.0%_BDM_Distortion -0.7
Int_Sb/Int_Sb5
Ground state: D_40.0%_BDM_Distortion -0.2
Int_Sb/Int_Sb2
Ground state: D_10.0%_BDM_Distortion -0.1
Int_Sb/Int_Sb9
Ground state: D_-10.0%_BDM_Distortion -0.4
Int_Sb/Int_Sb8
Ground state: D_-20.0%_BDM_Distortion -0.0
Int_Sb/Int_Sb1
Ground state: D_-10.0%_BDM_Distortion -1.1
Int_S/Int_S5
Ground state: D_-60.0%_BDM_Distortion -0.7
Int_S/Int_S4
Ground state: D_-30.0%_BDM_Distortion -0.6
Int_S/Int_S1
Ground state: D_-40.0%_BDM_Distortion -1.5
Int_S/Int_S6
Ground state: D_-0.0%_BDM_Distortion -0.0
Int_S/Int_S9
Ground state: D_30.0%_BDM_Distortion -1.8
S_Sb/S_Sb2
Ground state: D_-30.0%_BDM_Distortion -0.0
S_Sb/S_Sb1
Ground state: D_-20.0%_BDM_Distortion -0.0


In addition to using the root mean squared displacement (abbreviated as RMS) and the maximum distance between paired sites, we'll also use another metric that measures the similarity of site environments. The metric will be the cosine distance between the SOAP vectors of the defect sites. The SOAP vectors are a representation of the local environment around a given site. The cosine distance is a measure of how similar the two SOAP vectors are (e.g. how similar the site environments are). The smaller the distance, the more similar the environments.

In [10]:
cutoff = 5.0

for comp in energies:
    print("\nComposition", comp)
    print( "----------------")
    for defect in energies[comp]:
        print(defect)
        # Get distortion with lowest energy in the DFT/ground-truth results
        d = energies[comp][defect]["DFT"]["distortions"]
        d["Unperturbed"] = energies[comp][defect]["DFT"]["Unperturbed"]
        sorted_energies = dict(sorted(d.items(), key=lambda item: item[1]))
        gs_dist = list(sorted_energies.keys())[0]
        print("Ground state:", gs_dist, round(sorted_energies[gs_dist], 1))
        gs_struct_dft = structures[comp][defect]["DFT"][gs_dist]

        # Get final structure obtained with FF
        gs_struct_ff = structures[comp][defect]["MLFF"][gs_dist]
        # Get defect coordinates
        coords_dft = indentify_def_cor(gs_struct_dft, structure_perfect)
        coords_mlff = indentify_def_cor(gs_struct_ff, structure_perfect)

        # Add defect as last site and calculate SOAP vectors
        gs_struct_dft_copy = gs_struct_dft.copy()
        gs_struct_ff_copy = gs_struct_ff.copy()

        # Add ghost atom at the original position of the defect
        # This allows us to compare the environment around the defect in the DFT and FF structures
        gs_struct_dft_copy.append("V", coords_dft, coords_are_cartesian=False)
        gs_struct_ff_copy.append("V", coords_mlff, coords_are_cartesian=False)
        # To compare the site environment, we'll use SOAP. SOAP is a site descriptor that encodes the local environment
        # around a given point in the structure.
        # Calculate SOAPs
        soap_dft = get_soap_vec(gs_struct_dft_copy, r_cut=cutoff, n_max=10, l_max=10, sigma=1.0)
        soap_ff = get_soap_vec(gs_struct_ff_copy, r_cut=cutoff, n_max=10, l_max=10, sigma=1.0)
        # last vector is SOAP for the defect site (because it's the last site in the structure)
        cos_distance = cosine_distance(soap_dft[-1], soap_ff[-1])
        results[defect]["Cos. dist. SOAP (%)"] = round(cos_distance, 3)*100


Composition Sb2S3
----------------
Sb_S/Sb_S1
Ground state: D_-40.0%_BDM_Distortion -0.0
Sb_S/Sb_S3
Ground state: D_10.0%_BDM_Distortion -0.0
Sb_S/Sb_S2
Ground state: D_10.0%_BDM_Distortion -0.0
Int_Sb/Int_Sb3
Ground state: D_10.0%_BDM_Distortion -0.7
Int_Sb/Int_Sb4
Ground state: D_60.0%_BDM_Distortion -0.7
Int_Sb/Int_Sb5
Ground state: D_40.0%_BDM_Distortion -0.2
Int_Sb/Int_Sb2
Ground state: D_10.0%_BDM_Distortion -0.1
Int_Sb/Int_Sb9
Ground state: D_-10.0%_BDM_Distortion -0.4
Int_Sb/Int_Sb8
Ground state: D_-20.0%_BDM_Distortion -0.0
Int_Sb/Int_Sb1
Ground state: D_-10.0%_BDM_Distortion -1.1
Int_S/Int_S5
Ground state: D_-60.0%_BDM_Distortion -0.7
Int_S/Int_S4
Ground state: D_-30.0%_BDM_Distortion -0.6
Int_S/Int_S1
Ground state: D_-40.0%_BDM_Distortion -1.5
Int_S/Int_S6
Ground state: D_-0.0%_BDM_Distortion -0.0
Int_S/Int_S9
Ground state: D_30.0%_BDM_Distortion -1.8
S_Sb/S_Sb2
Ground state: D_-30.0%_BDM_Distortion -0.0
S_Sb/S_Sb1
Ground state: D_-20.0%_BDM_Distortion -0.0


In [11]:
energies[comp].keys()

dict_keys(['Sb_S/Sb_S1', 'Sb_S/Sb_S3', 'Sb_S/Sb_S2', 'Int_Sb/Int_Sb3', 'Int_Sb/Int_Sb4', 'Int_Sb/Int_Sb5', 'Int_Sb/Int_Sb2', 'Int_Sb/Int_Sb9', 'Int_Sb/Int_Sb8', 'Int_Sb/Int_Sb1', 'Int_S/Int_S5', 'Int_S/Int_S4', 'Int_S/Int_S1', 'Int_S/Int_S6', 'Int_S/Int_S9', 'S_Sb/S_Sb2', 'S_Sb/S_Sb1'])

In [12]:
# energies[comp]["Int_Sb/Int_Sb8"]["DFT"]

In [13]:
for host in structures:
    for defect in structures[host]:
        dis = results[defect]['GS distortion']
        # print(defect, dis_ml)
        mlff = energies[host][defect]['MLFF']['distortions'][dis]
        # print(mlff)
        dft = energies[host][defect]['DFT']['distortions'][dis]
        
        results[defect]['e_same_dis'] = abs(round(dft-mlff, 1))

In [14]:
# Show df
import pandas as pd
df = pd.DataFrame.from_dict(results, orient="index")
# Order by values in Cos. dist. SOAP (%)
df = df.sort_values(by=["Cos. dist. SOAP (%)"])
display(df)

,Defect,Host,GS distortion,Δ E (eV),Local RMS (A),Max. dist. (A),Cos. dist. SOAP (%),e_same_dis
Sb_S/Sb_S1,Sb_S/Sb_S1,Sb2S3,D_-40.0%_BDM_Distortion,-0.0,8.9,0.5,0.0,0.0
Int_S/Int_S9,Int_S/Int_S9,Sb2S3,D_30.0%_BDM_Distortion,-1.8,11.3,1.8,0.0,1.7
Int_S/Int_S6,Int_S/Int_S6,Sb2S3,D_-0.0%_BDM_Distortion,-0.0,4.1,0.5,0.0,0.5
Int_S/Int_S1,Int_S/Int_S1,Sb2S3,D_-40.0%_BDM_Distortion,-1.5,3.0,0.3,0.0,1.8
Int_Sb/Int_Sb1,Int_Sb/Int_Sb1,Sb2S3,D_-10.0%_BDM_Distortion,-1.1,9.0,0.8,0.0,1.1
S_Sb/S_Sb2,S_Sb/S_Sb2,Sb2S3,D_-30.0%_BDM_Distortion,-0.0,6.8,0.8,0.0,0.0
Int_Sb/Int_Sb9,Int_Sb/Int_Sb9,Sb2S3,D_-10.0%_BDM_Distortion,-0.4,5.8,0.5,0.0,0.1
Int_Sb/Int_Sb8,Int_Sb/Int_Sb8,Sb2S3,D_-20.0%_BDM_Distortion,-0.0,9.8,1.2,0.0,0.1
Int_Sb/Int_Sb5,Int_Sb/Int_Sb5,Sb2S3,D_40.0%_BDM_Distortion,-0.2,15.4,1.4,0.0,0.5
Int_Sb/Int_Sb3,Int_Sb/Int_Sb3,Sb2S3,D_10.0%_BDM_Distortion,-0.7,14.6,2.0,0.0,0.5


### 3.3.2 Find the MLFF structure that is more similar to the DFT structure

In [15]:
from copy import deepcopy

r_cutoff = 5.0  # Radial cutoff used when calculating the SOAP descriptor. We use 5A to focus on the local environment around the defect

results = {}
for comp in energies:
    print("\n\nComposition", comp)
    print("-------------------")
    for defect in energies[comp]:
        print(defect)
        # Get distortion with lowest energy
        d = energies[comp][defect]["DFT"]["distortions"]
        d["Unperturbed"] = energies[comp][defect]["DFT"]["Unperturbed"]
        sorted_energies = deepcopy(dict(sorted(d.items(), key=lambda item: item[1])))
        gs_dist = list(sorted_energies.keys())[0]
        # If energy lowering < -0.01, take Unperturbed as gs structure
        if (sorted_energies[gs_dist] - sorted_energies["Unperturbed"]) > -0.01:
            gs_dist = "Unperturbed"
        print("Ground state:", gs_dist, round(sorted_energies[gs_dist], 1))
        # print(structures[comp][defect]["DFT"])
        gs_struct_dft = structures[comp][defect]["DFT"][gs_dist]
        # Get final structure obtained with FF
        # print(structures[comp][defect]["MLFF"])
        gs_struct_ff = structures[comp][defect]["MLFF"][gs_dist]

        # Get defect coordinates
        coords_dft = indentify_def_cor(gs_struct_dft, structure_perfect)
        coords_mlff = indentify_def_cor(gs_struct_ff, structure_perfect)

        # Find closest MLFF structure to DFT ground state.
        # We do this by looping over the final MLFF structures, calculating the RMS to the
        # DFT ground state structure, and taking the MLFF structure with the lowest RMS.
        best_dist = gs_dist
        s1, s2 = get_local_structs(gs_struct_dft, gs_struct_ff, coords_dft, coords_mlff, r_cutoff)
        # s1, s2 = gs_struct_dft, gs_struct_ff # full structure
        best_rms = calc_rms_1_struct(s1, s2, metric="disp", stol=1.2, ltol=0.5, min_dist=0.12)
        max_dist = calc_rms_1_struct(s1, s2, metric="max_dist", stol=1.2, ltol=0.5, min_dist=0.12)
        for dist, struct in structures[comp][defect]["MLFF"].items():
            # Get local structs
            s1, s2 = get_local_structs(gs_struct_dft, struct, coords_dft, coords_mlff, r_cutoff)
            # Calculate local RMS
            rms = calc_rms_1_struct(s1, s2, metric="disp", stol=1.2, ltol=0.5, min_dist=0.12)
            distance = calc_rms_1_struct(s1, s2, metric="max_dist", stol=1.2, ltol=0.5, min_dist=0.12)
            if rms and best_rms: # if RMS lower than previous best, update best match (e.g. best RMS and best distortion)
                if rms < best_rms:
                    best_rms = rms
                    best_dist = dist
                    max_dist = distance

        results[defect] = {
            "Defect": defect,
            "Host": comp,
            "\u0394 E (eV)": round(sorted_energies[gs_dist], 1),
            "GS distortion": gs_dist,
            "Distortion leading to GS in FF": best_dist,
            "Local RMS (\u212B)": round(best_rms, 2) if best_rms is not None else None,
            "Max. dist. (\u212B)": round(max_dist, 1) if max_dist is not None else None,
        }



Composition Sb2S3
-------------------
Sb_S/Sb_S1
Ground state: Unperturbed 0.0
Sb_S/Sb_S3
Ground state: Unperturbed 0.0
Sb_S/Sb_S2
Ground state: D_10.0%_BDM_Distortion -0.0
Int_Sb/Int_Sb3
Ground state: D_10.0%_BDM_Distortion -0.7
Int_Sb/Int_Sb4
Ground state: D_60.0%_BDM_Distortion -0.7
Int_Sb/Int_Sb5
Ground state: D_40.0%_BDM_Distortion -0.2


3728928209.py:159: UserWarning: pymatgen StructureMatcher could not match lattices for structures.


Int_Sb/Int_Sb2
Ground state: D_10.0%_BDM_Distortion -0.1
Int_Sb/Int_Sb9
Ground state: D_-10.0%_BDM_Distortion -0.4
Int_Sb/Int_Sb8
Ground state: D_-20.0%_BDM_Distortion -0.0
Int_Sb/Int_Sb1
Ground state: D_-10.0%_BDM_Distortion -1.1
Int_S/Int_S5
Ground state: D_-60.0%_BDM_Distortion -0.7
Int_S/Int_S4
Ground state: D_-30.0%_BDM_Distortion -0.6
Int_S/Int_S1
Ground state: D_-40.0%_BDM_Distortion -1.5
Int_S/Int_S6
Ground state: D_-0.0%_BDM_Distortion -0.0
Int_S/Int_S9
Ground state: D_30.0%_BDM_Distortion -1.8
S_Sb/S_Sb2
Ground state: Unperturbed 0.0
S_Sb/S_Sb1
Ground state: Unperturbed 0.0


In [16]:
import pandas as pd

df = pd.DataFrame.from_dict(results, orient="index")
# Sort by Local RMS (A)
df = df.sort_values(by=["Local RMS (\u212B)"], ascending=True)
df

,Defect,Host,Δ E (eV),GS distortion,Distortion leading to GS in FF,Local RMS (Å),Max. dist. (Å)
S_Sb/S_Sb1,S_Sb/S_Sb1,Sb2S3,0.0,Unperturbed,D_20.0%_BDM_Distortion,1.42,0.3
Sb_S/Sb_S3,Sb_S/Sb_S3,Sb2S3,0.0,Unperturbed,D_-10.0%_BDM_Distortion,1.55,0.4
Sb_S/Sb_S1,Sb_S/Sb_S1,Sb2S3,0.0,Unperturbed,Unperturbed,1.73,0.4
Sb_S/Sb_S2,Sb_S/Sb_S2,Sb2S3,-0.0,D_10.0%_BDM_Distortion,D_-50.0%_BDM_Distortion,1.91,0.4
Int_S/Int_S6,Int_S/Int_S6,Sb2S3,-0.0,D_-0.0%_BDM_Distortion,D_60.0%_BDM_Distortion,2.65,0.4
Int_S/Int_S9,Int_S/Int_S9,Sb2S3,-1.8,D_30.0%_BDM_Distortion,D_10.0%_BDM_Distortion,3.55,1.8
Int_Sb/Int_Sb9,Int_Sb/Int_Sb9,Sb2S3,-0.4,D_-10.0%_BDM_Distortion,D_-60.0%_BDM_Distortion,4.24,0.6
S_Sb/S_Sb2,S_Sb/S_Sb2,Sb2S3,0.0,Unperturbed,Unperturbed,4.82,0.7
Int_S/Int_S4,Int_S/Int_S4,Sb2S3,-0.6,D_-30.0%_BDM_Distortion,Unperturbed,11.46,2.5
Int_S/Int_S5,Int_S/Int_S5,Sb2S3,-0.7,D_-60.0%_BDM_Distortion,D_10.0%_BDM_Distortion,12.74,1.4


Add SOAP vector as another metric to measure structural similarity:

In [17]:
cutoff = 5.0
for comp in energies:
    print("\nComposition", comp)
    print( "----------------")
    for defect in energies[comp]:
        print(defect)
        # Get distortion with lowest energy
        d = energies[comp][defect]["DFT"]["distortions"]
        d["Unperturbed"] = energies[comp][defect]["DFT"]["Unperturbed"]
        sorted_energies = dict(sorted(d.items(), key=lambda item: item[1]))
        gs_dist = list(sorted_energies.keys())[0]
        print("Ground state:", gs_dist, round(sorted_energies[gs_dist], 1))
        gs_struct_dft = structures[comp][defect]["DFT"][gs_dist]

        # Get final structure obtained with FF
        # gs_struct_ff = structures[comp][defect]["MLFF"][gs_dist]
        # Get defect coordinates
        coords_dft = indentify_def_cor(gs_struct_dft, structure_perfect)
        coords_mlff = indentify_def_cor(gs_struct_ff, structure_perfect)

        # Add defect as last site and calculate SOAP vectors
        gs_struct_dft_copy = gs_struct_dft.copy()
        # Get ff structure most similar to DFT gs, based on local RMS
        gs_dist_ff = results[defect]['Distortion leading to GS in FF']
        gs_struct_ff_copy = structures[comp][defect]["MLFF"][gs_dist_ff].copy()
        # Add vacancy
        gs_struct_dft_copy.append("V", coords_dft, coords_are_cartesian=False)
        gs_struct_ff_copy.append("V", coords_mlff, coords_are_cartesian=False)

        # Calculate SOAPs
        soap_dft = get_soap_vec(gs_struct_dft_copy, r_cut=cutoff, n_max=10, l_max=10, sigma=1.0)
        soap_ff = get_soap_vec(gs_struct_ff_copy, r_cut=cutoff, n_max=10, l_max=10, sigma=1.0)
        cos_distance = cosine_distance(soap_dft[-1], soap_ff[-1]) # last vector is SOAP for the defect
        results[defect]["Cos. dist. SOAP (%)"] = round(cos_distance, 3)*100


Composition Sb2S3
----------------
Sb_S/Sb_S1
Ground state: D_-40.0%_BDM_Distortion -0.0
Sb_S/Sb_S3
Ground state: D_10.0%_BDM_Distortion -0.0
Sb_S/Sb_S2
Ground state: D_10.0%_BDM_Distortion -0.0
Int_Sb/Int_Sb3
Ground state: D_10.0%_BDM_Distortion -0.7
Int_Sb/Int_Sb4
Ground state: D_60.0%_BDM_Distortion -0.7
Int_Sb/Int_Sb5
Ground state: D_40.0%_BDM_Distortion -0.2
Int_Sb/Int_Sb2
Ground state: D_10.0%_BDM_Distortion -0.1
Int_Sb/Int_Sb9
Ground state: D_-10.0%_BDM_Distortion -0.4
Int_Sb/Int_Sb8
Ground state: D_-20.0%_BDM_Distortion -0.0
Int_Sb/Int_Sb1
Ground state: D_-10.0%_BDM_Distortion -1.1
Int_S/Int_S5
Ground state: D_-60.0%_BDM_Distortion -0.7
Int_S/Int_S4
Ground state: D_-30.0%_BDM_Distortion -0.6
Int_S/Int_S1
Ground state: D_-40.0%_BDM_Distortion -1.5
Int_S/Int_S6
Ground state: D_-0.0%_BDM_Distortion -0.0
Int_S/Int_S9
Ground state: D_30.0%_BDM_Distortion -1.8
S_Sb/S_Sb2
Ground state: D_-30.0%_BDM_Distortion -0.0
S_Sb/S_Sb1
Ground state: D_-20.0%_BDM_Distortion -0.0


In [18]:
import pandas as pd

df = pd.DataFrame.from_dict(results, orient="index")
# Sort by Local RMS (A)
df = df.sort_values(by=['Cos. dist. SOAP (%)'], ascending=True)
display(df)

,Defect,Host,Δ E (eV),GS distortion,Distortion leading to GS in FF,Local RMS (Å),Max. dist. (Å),Cos. dist. SOAP (%)
S_Sb/S_Sb1,S_Sb/S_Sb1,Sb2S3,0.0,Unperturbed,D_20.0%_BDM_Distortion,1.42,0.3,0.0
Int_S/Int_S1,Int_S/Int_S1,Sb2S3,-1.5,D_-40.0%_BDM_Distortion,D_-40.0%_BDM_Distortion,NaN,NaN,0.0
Int_Sb/Int_Sb3,Int_Sb/Int_Sb3,Sb2S3,-0.7,D_10.0%_BDM_Distortion,D_-0.0%_BDM_Distortion,12.83,2.0,0.0
Int_Sb/Int_Sb1,Int_Sb/Int_Sb1,Sb2S3,-1.1,D_-10.0%_BDM_Distortion,D_-10.0%_BDM_Distortion,NaN,NaN,0.0
Int_S/Int_S4,Int_S/Int_S4,Sb2S3,-0.6,D_-30.0%_BDM_Distortion,Unperturbed,11.46,2.5,0.1
Int_S/Int_S5,Int_S/Int_S5,Sb2S3,-0.7,D_-60.0%_BDM_Distortion,D_10.0%_BDM_Distortion,12.74,1.4,0.1
Sb_S/Sb_S1,Sb_S/Sb_S1,Sb2S3,0.0,Unperturbed,Unperturbed,1.73,0.4,0.1
Int_Sb/Int_Sb4,Int_Sb/Int_Sb4,Sb2S3,-0.7,D_60.0%_BDM_Distortion,D_-0.0%_BDM_Distortion,33.60,2.7,0.1
Sb_S/Sb_S2,Sb_S/Sb_S2,Sb2S3,-0.0,D_10.0%_BDM_Distortion,D_-50.0%_BDM_Distortion,1.91,0.4,0.1
Int_Sb/Int_Sb2,Int_Sb/Int_Sb2,Sb2S3,-0.1,D_10.0%_BDM_Distortion,D_10.0%_BDM_Distortion,NaN,NaN,0.1


In [19]:
print(energies['Sb2S3']['Sb_S/Sb_S1']['MLFF'])

{'distortions': {'D_30.0%_BDM_Distortion': 0.025482177734375, 'D_-40.0%_BDM_Distortion': 0.031524658203125, 'D_60.0%_BDM_Distortion': 0.13134765625, 'D_-20.0%_BDM_Distortion': 0.308197021484375, 'D_-10.0%_BDM_Distortion': -0.000152587890625, 'D_50.0%_BDM_Distortion': 0.034454345703125, 'D_40.0%_BDM_Distortion': 0.025482177734375, 'D_-0.0%_BDM_Distortion': 0.031524658203125, 'D_-30.0%_BDM_Distortion': 0.308197021484375, 'D_-50.0%_BDM_Distortion': 0.308197021484375, 'D_10.0%_BDM_Distortion': 0.031524658203125, 'D_20.0%_BDM_Distortion': 0.04205322265625, 'D_-60.0%_BDM_Distortion': 0.308197021484375}, 'Unperturbed': 0.0}


In [21]:
#### writing cif. file
import os
from pymatgen.core.structure import Structure

def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def cif_writer(defect, dis_dft, dis_mlff, structures):
    ensure_dir('cif/')
    defect_filename = defect.replace('/', '_')
    
    # Generate CIF for DFT
    s_DFT = structures["Sb2S3"][defect]['DFT'][dis_dft].copy()
    coords_dft = indentify_def_cor(s_DFT, structure_perfect)
    s_DFT.append("V", coords_dft, coords_are_cartesian=False)
    s_DFT.to(fmt="cif", filename=f'{defect_filename}_{dis_dft}_dft.cif')
    
    # Generate CIF for MLFF
    s_MLFF = structures["Sb2S3"][defect]['DFT'][dis_mlff].copy()
    coords_mlff = indentify_def_cor(s_MLFF, structure_perfect)
    s_MLFF.append("V", coords_mlff, coords_are_cartesian=False)
    s_MLFF.to(fmt="cif", filename=f'{defect_filename}_{dis_mlff}_mlff.cif')


for host in structures:
    for defect in structures[host]:
        dis_dft = results[defect]['GS distortion']
        dis_mlff = results[defect]['Distortion leading to GS in FF']
        
        cif_writer(defect, dis_dft, dis_mlff, structures)


In [22]:
for host in structures:
    for defect in structures[host]:
        dis_dft = results[defect]['GS distortion']
        dis_ml = results[defect]['Distortion leading to GS in FF']
        # print(defect, dis_ml)
        if dis_ml == 'Unperturbed':
            mlff = 0.0
        else: 
            mlff = energies[host][defect]['MLFF']['distortions'][dis_ml]
        # print(mlff)
        dft = energies[host][defect]['DFT']['distortions'][dis_dft]
        
        results[defect]['e_soap'] = round(dft-mlff, 1)
        


In [23]:
df = pd.DataFrame.from_dict(results, orient="index")
# Sort by Local RMS (A)
df = df.sort_values(by=['Cos. dist. SOAP (%)'], ascending=True)
display(df)

,Defect,Host,Δ E (eV),GS distortion,Distortion leading to GS in FF,Local RMS (Å),Max. dist. (Å),Cos. dist. SOAP (%),e_soap
S_Sb/S_Sb1,S_Sb/S_Sb1,Sb2S3,0.0,Unperturbed,D_20.0%_BDM_Distortion,1.42,0.3,0.0,0.2
Int_S/Int_S1,Int_S/Int_S1,Sb2S3,-1.5,D_-40.0%_BDM_Distortion,D_-40.0%_BDM_Distortion,NaN,NaN,0.0,-1.8
Int_Sb/Int_Sb3,Int_Sb/Int_Sb3,Sb2S3,-0.7,D_10.0%_BDM_Distortion,D_-0.0%_BDM_Distortion,12.83,2.0,0.0,-0.4
Int_Sb/Int_Sb1,Int_Sb/Int_Sb1,Sb2S3,-1.1,D_-10.0%_BDM_Distortion,D_-10.0%_BDM_Distortion,NaN,NaN,0.0,-1.1
Int_S/Int_S4,Int_S/Int_S4,Sb2S3,-0.6,D_-30.0%_BDM_Distortion,Unperturbed,11.46,2.5,0.1,-0.6
Int_S/Int_S5,Int_S/Int_S5,Sb2S3,-0.7,D_-60.0%_BDM_Distortion,D_10.0%_BDM_Distortion,12.74,1.4,0.1,-0.4
Sb_S/Sb_S1,Sb_S/Sb_S1,Sb2S3,0.0,Unperturbed,Unperturbed,1.73,0.4,0.1,0.0
Int_Sb/Int_Sb4,Int_Sb/Int_Sb4,Sb2S3,-0.7,D_60.0%_BDM_Distortion,D_-0.0%_BDM_Distortion,33.60,2.7,0.1,-0.9
Sb_S/Sb_S2,Sb_S/Sb_S2,Sb2S3,-0.0,D_10.0%_BDM_Distortion,D_-50.0%_BDM_Distortion,1.91,0.4,0.1,-0.0
Int_Sb/Int_Sb2,Int_Sb/Int_Sb2,Sb2S3,-0.1,D_10.0%_BDM_Distortion,D_10.0%_BDM_Distortion,NaN,NaN,0.1,-0.1


In [31]:
defect_coords = {}
for comp in structures:
    print("Composition", comp)
    defect_coords[comp] = {}
    for defect in structures[comp]:
        # Get fractional coordinates of defect
        defect_coords[comp][defect] = indentify_def_cor(structures[comp][defect]["DFT"]['Unperturbed'], structure_perfect)

Composition Sb2S3


In [33]:
from copy import deepcopy
from pymatgen.core.structure import Structure
from collections import defaultdict
# comp = 'Sb2Se3'
# defect_no_charge = defect.rsplit("_", 1)[0]
# coords = defect_coords[comp][defect_no_charge]
def get_num_diff_structures_soap(
    dict_structs,
    defect_coords: list,
    threshold=0.01,
    cutoff=5.0
):
    """Get number of local minima for a given defect.
    Args:
        dict_structs: Dictionary of structures to compare. Should contain keys with
            the distortions and the associated (relaxed) structures.
        defect_coords: Fractinal oordinates of the defect.
        threshold: Threshold to consider two structures different.
        cutoff: Cutoff radius for SOAP.
    
   Returns: number of unique structures/local minima and list with unique distortions
   """
    # Add defect as last site and calculate SOAP vectors
    strucs = deepcopy(dict_structs)
    strucs = {
        k: v.append("V", defect_coords, coords_are_cartesian=False)
        for k, v in strucs.items() if type(v) == Structure
    }
    # Calculate SOAPs of (original) defect site for all structures/distortions
    soap_vecs = {
        k: get_soap_vec(s, r_cut=cutoff, n_max=10, l_max=10, sigma=1.0)[-1]
        for k, s in strucs.items()
    }
    # Calculate cosine distance between the SOAPs for each distortion and Unperturbed
    cos_distances = {
        k: cosine_distance(soap_vecs["Unperturbed"], v) for k, v in soap_vecs.items()
    }
    structs_diff_to_unp = {k: v for k, v in cos_distances.items() if v > threshold}
    if len(structs_diff_to_unp) < 2:
        # If there is only 1 or less distortions different from the unperturbed, the defect has either
        # a single or two local minima
        num_local_minima = 1 + len(structs_diff_to_unp)
        unique_structs = list(structs_diff_to_unp.keys())
        unique_structs.append("Unperturbed")
    else:  # Compare the structures between them
        unique_structs = ["Unperturbed", list(structs_diff_to_unp.keys())[0],]
        num_local_minima = 2
        for distortion in list(structs_diff_to_unp.keys())[1:]:
            # print("\nDistortion", dist)
            different = True
            for distortion_2 in unique_structs:
                # print("Comparing to struct", distortion)
                cos = cosine_distance(soap_vecs[distortion], soap_vecs[distortion_2])
                if cos < threshold:  # If cosine distance is below threshold, the distortions are the same
                    different = False
                    # print("Breaking")
                    break
                # print("Different", different)
            if different:  # If the distortion is different from all the others, add to list of unique structures
                unique_structs.append(distortion)
                num_local_minima += 1
    return num_local_minima, unique_structs
# Apply function
# Assuming you have a structures dictionary with keys for the composition and the defect, and the associated relaxed structures
# structures = {composition: {
    # defect1: {"DFT": {"Unperturbed": Structure, "Dist1": Structure, ...}, "FF": Structure},
    # defect2: {"DFT": {...}, "FF": {...}},
    # ...}, ...
threshold = 0.0001 # Distortions whose structures have a SOAP difference greater than this value are considered different
results_minima = defaultdict(dict)
comp = ['Sb2Se3']
for comp in energies:
    print("Composition", comp)
    for defect in energies[comp]:
        # Get fractional coordinates of defect
        coords = defect_coords[comp][defect]
        # Number of unique structures identified with DFT
        num_unique_structs_dft, unique_structs_dft = get_num_diff_structures_soap(
            structures[comp][defect]["DFT"],
            defect_coords=coords,
            threshold=threshold
        )
        # Number of unique structures identified with FF
        num_unique_structs_ff, unique_structs_ff = get_num_diff_structures_soap(
            structures[comp][defect]["MLFF"],
            defect_coords=coords,
            threshold=threshold
        )
        try:
            print(f"Current defect: {defect}")
            results_minima[defect]["Num. local minima FF|DFT"] = f"{num_unique_structs_ff} | {num_unique_structs_dft}"
            results_minima[defect]["Defect"] = defect
        except KeyError:
            print(f"Key error for {defect}")
            continue
# Display results
df_minima = pd.DataFrame.from_dict(results_minima, orient="index")
# Sort by Cos. dist. SOAP (%)
df_merged = pd.merge(df, df_minima, on='Defect', how='left')
df_local_minima = df_merged.sort_values(by=["Cos. dist. SOAP (%)"])
display(df_local_minima)

Composition Sb2S3
Current defect: Sb_S/Sb_S1
Current defect: Sb_S/Sb_S3
Current defect: Sb_S/Sb_S2
Current defect: Int_Sb/Int_Sb3
Current defect: Int_Sb/Int_Sb4
Current defect: Int_Sb/Int_Sb5
Current defect: Int_Sb/Int_Sb2
Current defect: Int_Sb/Int_Sb9
Current defect: Int_Sb/Int_Sb8
Current defect: Int_Sb/Int_Sb1
Current defect: Int_S/Int_S5
Current defect: Int_S/Int_S4
Current defect: Int_S/Int_S1
Current defect: Int_S/Int_S6
Current defect: Int_S/Int_S9
Current defect: S_Sb/S_Sb2
Current defect: S_Sb/S_Sb1


,Defect,Host,Δ E (eV),GS distortion,Distortion leading to GS in FF,Local RMS (Å),Max. dist. (Å),Cos. dist. SOAP (%),e_soap,Num. local minima FF|DFT
0,S_Sb/S_Sb1,Sb2S3,0.0,Unperturbed,D_20.0%_BDM_Distortion,1.42,0.3,0.0,0.2,7 | 6
1,Int_S/Int_S1,Sb2S3,-1.5,D_-40.0%_BDM_Distortion,D_-40.0%_BDM_Distortion,NaN,NaN,0.0,-1.8,5 | 6
2,Int_Sb/Int_Sb3,Sb2S3,-0.7,D_10.0%_BDM_Distortion,D_-0.0%_BDM_Distortion,12.83,2.0,0.0,-0.4,6 | 13
3,Int_Sb/Int_Sb1,Sb2S3,-1.1,D_-10.0%_BDM_Distortion,D_-10.0%_BDM_Distortion,NaN,NaN,0.0,-1.1,3 | 9
10,Int_Sb/Int_Sb8,Sb2S3,-0.0,D_-20.0%_BDM_Distortion,D_-50.0%_BDM_Distortion,77.99,1.8,0.1,0.1,5 | 11
9,Int_Sb/Int_Sb2,Sb2S3,-0.1,D_10.0%_BDM_Distortion,D_10.0%_BDM_Distortion,NaN,NaN,0.1,-0.1,2 | 6
7,Int_Sb/Int_Sb4,Sb2S3,-0.7,D_60.0%_BDM_Distortion,D_-0.0%_BDM_Distortion,33.60,2.7,0.1,-0.9,6 | 8
8,Sb_S/Sb_S2,Sb2S3,-0.0,D_10.0%_BDM_Distortion,D_-50.0%_BDM_Distortion,1.91,0.4,0.1,-0.0,4 | 10
5,Int_S/Int_S5,Sb2S3,-0.7,D_-60.0%_BDM_Distortion,D_10.0%_BDM_Distortion,12.74,1.4,0.1,-0.4,2 | 6
4,Int_S/Int_S4,Sb2S3,-0.6,D_-30.0%_BDM_Distortion,Unperturbed,11.46,2.5,0.1,-0.6,4 | 7
